In [6]:
import os
import re
import numpy as np
import matplotlib
import math
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

# python3 -m pip install --user buildingspy
from buildingspy.io.outputfile import Reader
from buildingspy.io.postprocess import Plotter

def loom(series, span, tol):
    time = series[0]
    temp = series[1]
    weaver = []
    num_step = 0
    sum_temp = 0
    for i in range(len(time)):
        if time[i] < span * (len(weaver) + 1) + tol:
            num_step = num_step + 1
            sum_temp = sum_temp + temp[i]
        else:
            weaver.append(sum_temp / num_step)
            num_step = 0
            sum_temp = 0
        # append the last hour when iteration is over
        if i == len(time) - 1:
            weaver.append(sum_temp / num_step)
    return weaver

def offset_val(series):
    return [val - 273.15 for val in series]

def get_unmet_hours(res_reader, temp_span, work_span):
    tags = []
    unmethours = []
    for tag in r.varNames('TAir'):
        checker = re.search('TAir$', tag)
        if (checker != None):
            (t, y) = r.values(tag)
            unmethour = 0
            temps = loom((t, y), 3600, 10)
            temps = offset_val(temps)
            for i in range(len(temps)):
                if temps[i] < temp_span[0] or temps[i] > temp_span[1]:  # within the temp span
                    if i%24 >= work_span[0] and i%24 < work_span[1]:    # within the working hours
                        unmethour = unmethour + 1
            unmethours.append(unmethour)
            tags.append(tag)
    return dict(zip(tags, unmethours))

def get_fan_utility(res_reader):
    utility = []
    for tag in r.varNames('fan'):
        checker = re.search('.P$', tag)
        if (checker != None and len(tag.split('.')) == 2):
            print(f"fan: {tag}")
            utility.append(r.integral(tag) / 3600000)
    return utility

def get_exc_utility(res_reader):
    utility = []
    for tag in r.varNames('exc'):
        checker = re.search('.Q_flow$', tag)
        if (checker != None and len(tag.split('.')) == 2):
            print(f"exc: {tag}")
            utility.append(r.integral(tag) / 3600000)
    return utility

# drawing
def plot_air_temps(res_reader):
    for tag in r.varNames('TAir'):
        checker = re.search('TAir$', tag)
        if (checker != None):
            (t, y) = r.values(tag)
            temps = loom((t, y), 3600, 10)                  # average series data per hour
            temps = offset_val(temps)
            times = [i * 3600 for i in range(len(temps))]
            tSup = np.linspace(0, 3600*len(temps), num=len(temps))
            TInt = Plotter.interpolate(tSup, times, temps)
            plt.plot(tSup, TInt)
    plt.show()

def temp_scaler(res_reader, temp_set, work_span):
    scalers = []
    names = []
    for tag in r.varNames('TAir'):
        checker = re.search('TAir$', tag)
        if (checker != None):
            names.append(tag)
            (t, y) = r.values(tag)  
            temps = loom((t, y), 3600, 10)
            temps = offset_val(temps)
            baseline = 0
            temp_samples = []
            for i in range(len(temps)):
                if i%24 >= work_span[0] and i%24 < work_span[1]:
                    temp_samples.append(temps[i])
                if i%24 == 5:
                    baseline = temps[i]
            print(temp_samples)
            temp_mean = sum(temp_samples) / len(temp_samples)
            temp_ratio = (temp_set - temp_mean) / (temp_mean - baseline)
            # scalers.append(2 * temp_ratio + 1)
            scalers.append(temp_ratio / math.sqrt(abs(temp_ratio)) + 1)
    return scalers, names


# Read data
resultFile = r"C:\Users\ianqu\AppData\Local\Temp\OpenModelica\OMEdit\Angelo_1\Angelo_1_res.mat"
r=Reader(resultFile, "openmodelica")

# presets
# scalers, names = temp_scaler(r, 21, (9, 18))
# run
# plot_air_temps(r)
# get_unmet_hours(r, (20, 22), (9, 18))
print(sum(get_unmet_hours(r, (20, 27), (9, 18)).values()))
print(sum(get_fan_utility(r)))
print(get_exc_utility(r))



8004
267.2483999836287
[691.5774336983565, 1616.10000321776, 1513.0910442484105, 1094.3870807002188, 2538.5269205492436, 2385.612409148287, 270.17356251026695, 1763.6446970295235]


In [ ]:
import OMPython
help(OMPython)

In [ ]:
from OMPython import OMCSessionZMQ
omc = OMCSessionZMQ()
omc.sendExpression("loadModel(Modelica)")
omc.sendExpression("loadFile(\"C:/Users/ianqu/Documents/Buildings 9.1.0/package.mo\")")
omc.sendExpression("loadFile(\"C:/Users/ianqu/Desktop/clams/Angelo/Angelo.mo\")")
omc.sendExpression("simulate(Buildings.ThermalZones.EnergyPlus_9_6_0.Angelo)")